In [21]:
import pandas as pd
import plotly.express as px
import plotly.colors as pc

# --- Đọc dữ liệu ---
hr = pd.read_csv("hr_vn_sim.csv")

# --- Chọn các biến định lượng ---
cols = ["Age", "YearsAtCompany", "JobSatisfaction", "WorkLifeBalance", "IncomeClass"]

hr = hr.dropna(subset=cols)

light_viridis = [
    pc.label_rgb(tuple(int(c[i:i+2], 16) for i in (1, 3, 5)) + (100,))
    for c in px.colors.sequential.Viridis
]

fig = px.parallel_coordinates(
    hr,
    dimensions=cols,
    color="Attrition_prob",
    color_continuous_scale=px.colors.sequential.Viridis_r,  # đảo màu
    labels={c: c.replace("_", " ") for c in cols},
    title="Parallel Coordinate Plot – Hồ sơ nhân viên (thang màu đảo)"
)


# --- Tùy chỉnh giao diện ---
fig.update_layout(
    title_font_size=18,
    title_x=0.5,
    plot_bgcolor='white'
)

fig.write_html("parallel_plot.html")
import webbrowser
webbrowser.open("parallel_plot.html")

True

In [6]:
import pandas as pd 
import altair as alt 
hr = pd.read_csv("hr_vn_sim.csv")

# --- Chọn các biến cần trực quan hóa ---
cols = ["IncomeClass","Age", "WorkLifeBalance", "Attrition_prob", "YearsAtCompany", "JobSatisfaction"] 
hr = hr[cols].dropna()

# --- Chuyển dữ liệu sang dạng "long" để Altair hiểu Parallel Coordinates ---
hr_long = hr.melt(
    id_vars="Attrition_prob",
    var_name="Variable",
    value_name="Value"
)

# --- Parallel Coordinates Plot với α-blending (làm mờ để tránh overplotting) ---
chart = (
    alt.Chart(hr_long)
    .mark_line(opacity=0.1)
    .encode(
        x=alt.X("Variable:N", title="Biến"),
        y=alt.Y("Value:Q", title="Giá trị (chuẩn hóa)"),
        color=alt.Color(
            "Attrition_prob:Q", 
            scale=alt.Scale(scheme="viridis"), 
            legend=alt.Legend(title="Xác suất nghỉ việc")
        ),
        detail="Attrition_prob"  # mỗi dòng = 1 cá nhân
    )
    .properties(
        width=700,
        height=400,
        title="Parallel Coordinate Plot – α-blending theo Attrition Probability"
    )
)

chart.interactive()

alt.Chart(...)

In [2]:
import pandas as pd
import altair as alt

# --- Đọc dữ liệu ---
hr = pd.read_csv("hr_vn_sim.csv")
cols = ["Age", "YearsAtCompany", "JobSatisfaction", "WorkLifeBalance", "IncomeClass", "Attrition_prob"]
hr = hr[cols].dropna()

# --- Chuẩn hóa Min–Max ---
hr_norm = hr.copy()
for c in cols[:-1]:
    hr_norm[c] = (hr[c] - hr[c].min()) / (hr[c].max() - hr[c].min())

# --- Sắp xếp trục theo logic: Tuổi → Thâm niên → Thu nhập → Thỏa mãn → Cân bằng ---
order = ["Age", "YearsAtCompany", "IncomeClass", "JobSatisfaction", "WorkLifeBalance"]

# --- Dạng "long" ---
hr_long = hr_norm.melt(id_vars="Attrition_prob", var_name="Variable", value_name="Value")

chart_minmax = alt.Chart(hr_long).mark_line(opacity=0.1).encode(
    x=alt.X("Variable:N", sort=order, title="Biến"),
    y=alt.Y("Value:Q", title="Giá trị (chuẩn hóa Min–Max)"),
    color=alt.Color("Attrition_prob:Q", scale=alt.Scale(scheme="viridis")),
    detail="Attrition_prob"
).properties(
    width=600,
    height=400,
    title="PCP – Chuẩn hóa Min–Max"
)
chart_minmax


alt.Chart(...)

In [7]:
hr_z = hr.copy()
for c in cols[:-1]:
    hr_z[c] = (hr[c] - hr[c].mean()) / hr[c].std()

hr_long_z = hr_z.melt(id_vars="Attrition_prob", var_name="Variable", value_name="Value")

chart_z = alt.Chart(hr_long_z).mark_line(opacity=0.15).encode(
    x=alt.X("Variable:N", sort=order, title="Biến"),
    y=alt.Y("Value:Q", title="(Giá trị - mean)/std"),
    color=alt.Color("Attrition_prob:Q", scale=alt.Scale(scheme="plasma")),
    detail="Attrition_prob"
).properties(
    width=600,
    height=400,
    title="PCP – Chuẩn hóa theo Z-score"
)
chart_z


alt.Chart(...)

In [27]:
import pandas as pd
import altair as alt

# --- Đọc dữ liệu ---
hr = pd.read_csv("hr_vn_sim.csv")

# --- Chọn các biến ---
cols = ["Age", "YearsAtCompany", "JobSatisfaction", "WorkLifeBalance", "IncomeClass", "Attrition_prob"]
hr = hr[cols].dropna()

# --- Scatterplot Matrix (SPLOM) ---
splom = alt.Chart(hr).mark_point(
    opacity=0.45,       # α-blending nhẹ
    filled=True          # điểm tô màu đặc
).encode(
    alt.X(alt.repeat("column"), type="quantitative"),
    alt.Y(alt.repeat("row"), type="quantitative"),
    color=alt.Color(
        "Attrition_prob:Q",
        scale=alt.Scale(scheme="turbo"),   # 🔥 Thay đổi scheme tại đây ("Plasma", "Inferno", "Spectral", ...)
        legend=alt.Legend(title="Attrition Probability")
    ),
    tooltip=["Age", "YearsAtCompany", "JobSatisfaction", "WorkLifeBalance", "IncomeClass", "Attrition_prob"]
).properties(
    width=110,
    height=110,
    title="Scatterplot Matrix (SPLOM) – Highlight Attrition Probability"
).repeat(
    row=cols[:-1],
    column=cols[:-1]
).interactive()

splom


alt.RepeatChart(...)

In [23]:
import pandas as pd
import altair as alt
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import MDS

# --- Đọc dữ liệu ---
hr = pd.read_csv("hr_vn_sim.csv")

# --- Các biến định lượng để dùng trong MDS ---
cols = ["Age", "YearsAtCompany", "JobSatisfaction", "WorkLifeBalance", "IncomeClass"]
hr = hr.dropna(subset=cols)

# --- Chuẩn hóa dữ liệu ---
X = StandardScaler().fit_transform(hr[cols])

# --- Giảm chiều bằng MDS ---
mds = MDS(n_components=2, random_state=42, dissimilarity='euclidean')
X_2d = mds.fit_transform(X)

# --- Gộp kết quả vào DataFrame ---
hr["Dim1"] = X_2d[:, 0]
hr["Dim2"] = X_2d[:, 1]

# --- Brush để chọn vùng trên MDS ---
brush = alt.selection_interval(encodings=['x', 'y'])

# --- Biểu đồ MDS ---
mds_plot = alt.Chart(hr).mark_circle(size=60).encode(
    x="Dim1",
    y="Dim2",
    color=alt.condition(brush, "Attrition:N", alt.value("lightgray")),
    tooltip=["Department", "Age", "YearsAtCompany", "IncomeClass", "Attrition"]
).add_params(
    brush
).properties(
    width=400, height=350,
    title="MDS: Biểu diễn nhân viên trong không gian 2 chiều"
)

# --- PCP (Parallel Coordinates) ---
hr_long = hr.melt(id_vars=["Attrition", "Dim1", "Dim2"],
                  value_vars=cols,
                  var_name="Variable",
                  value_name="Value")

pcp = alt.Chart(hr_long).mark_line(opacity=0.2).encode(
    x="Variable:N",
    y="Value:Q",
    color="Attrition:N",
    detail="Attrition"
).transform_filter(
    brush
).properties(
    width=400, height=350,
    title="Parallel Coordinate Plot – Hồ sơ nhóm được chọn"
)

# --- Hiển thị song song ---
final_chart = mds_plot | pcp
final_chart


alt.HConcatChart(...)

In [28]:
import seaborn as sns
import pandas as pd
import altair as alt

# --- Load dữ liệu ---
mpg = sns.load_dataset("mpg").dropna()

# --- Chọn biến ---
cols = ["mpg", "horsepower", "weight", "acceleration", "displacement", "origin"]

# --- Parallel Coordinates Plot ---
mpg_long = mpg.melt(id_vars="origin", value_vars=cols[:-1], var_name="Variable", value_name="Value")

pcp = alt.Chart(mpg_long).mark_line(opacity=0.2).encode(
    x=alt.X("Variable:N", title="Thuộc tính xe"),
    y=alt.Y("Value:Q", title="Giá trị"),
    color=alt.Color("origin:N", legend=alt.Legend(title="Xuất xứ"))
).properties(
    width=600,
    height=350,
    title="Parallel Coordinates Plot – Phân biệt xe theo xuất xứ"
)

# --- Scatterplot Matrix ---
splom = alt.Chart(mpg).mark_point(opacity=0.4, filled=True, size=30).encode(
    x=alt.X(alt.repeat("column"), type="quantitative"),
    y=alt.Y(alt.repeat("row"), type="quantitative"),
    color=alt.Color("origin:N", legend=alt.Legend(title="Xuất xứ")),
    tooltip=["mpg", "horsepower", "weight", "acceleration", "origin"]
).properties(
    width=110,
    height=110,
    title="Scatterplot Matrix (SPLOM) – Mối quan hệ giữa các biến"
).repeat(
    row=cols[:-1],
    column=cols[:-1]
).interactive()

pcp & splom


alt.VConcatChart(...)